In [1]:
from pathlib import Path
import pandas as pd

In [2]:
p_src = Path('../data/v0.1.0.books_predictions.csv')
p_dest = Path('../data/v0.1.0.final.csv')

In [3]:
df_src = pd.read_csv(p_src)

In [4]:
df_dest = pd.DataFrame(index=df_src.index)

for col in ['id', 'title', 'author']:
    df_dest[col] = df_src[col]

df_dest['human_categories'] = df_src['categories']
df_dest['ai_categories'] = df_src['pred_categories']

def calculate_minus(series_base: pd.Series, series_to_subtract: pd.Series) -> pd.Series:
    res = series_base.copy()
    for i, (v_base, v_to_subtract) in enumerate(zip(series_base, series_to_subtract)):
        res[i] = '|'.join(set(v_base.split('|') if not pd.isna(v_base) else []) - set(v_to_subtract.split('|') if not pd.isna(v_to_subtract) else []))
    return res

df_dest['ai_added_categories'] = calculate_minus(df_dest['ai_categories'], df_dest['human_categories'])
df_dest['human_added_categories'] = calculate_minus(df_dest['human_categories'], df_dest['ai_categories'])

for col in ['overview', 'author_overview', 'pred_reasoning_steps']:
    df_dest[col] = df_src[col]

In [5]:
df_dest.head(5)

,id,title,author,human_categories,ai_categories,ai_added_categories,human_added_categories,overview,author_overview,pred_reasoning_steps
0,0059543c-a8e5-4615-a78c-69f8e73c19c1,Parenting the New Teen in the Age of Anxiety: ...,John Duffy,Family,Family|Health|Society & Tech|Self-Growth,Health|Society & Tech|Self-Growth,,You're looking at a modern parenting guide. Te...,John Duffy is a successful clinical psychologi...,"The book is a parenting guide, which directly ..."
1,00bb4531-5150-44af-9eb1-c160c7d26ed6,"Right Thing, Right Now: Good Values, Good Char...",Ryan Holiday,Self-Growth,Happiness|Spirituality|Self-Growth,Happiness|Spirituality,,The modern world has many fascinating inventio...,Ryan Holiday is known for bringing philosophy ...,The book discusses the concept of happiness in...
2,012399fc36f74c489e4e,Eat Move Sleep: How Small Choices Lead to Big ...,Tom Rath,Health,Health|Self-Growth|Productivity,Productivity|Self-Growth,,"Discover how inner strength, willpower, and ri...",Tom Rath is a renowned author of best-selling ...,"The book focuses on well-being, which is close..."
3,012d6fde-51e2-4289-822a-1cbb57394bcd,"From Strength to Strength: Finding Success, Ha...",Arthur C. Brooks,Happiness|Self-Growth,Happiness|Self-Growth,,,"Achieving your dreams is fulfilling, but what ...",Arthur C. Brooks is a scientist whose research...,"The book focuses on finding purpose, joy, and ..."
4,017dab0f35d34af59653,"Dream Big: Know What You Want, Why You Want It...",Bob Goff,Self-Growth,Self-Growth|Productivity,Productivity,,Break free from the cycle of unfinished projec...,"Robert Kendall Goff is a speaker, lawyer, auth...","The book's title, 'Dream Big', suggests a focu..."


In [6]:
df_dest.to_csv(p_dest, index=None)